In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic",set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [3]:
from langchain_core.prompts import PromptTemplate

### 1. llm에 질문하는 프롬프트 만들기
1. 변수가 들어가는 문자열 변수 생성
2. PromptTemplate.from_template으로 PromptTemplate 객체생성
2-1. PromptTemplate 객체는 변수에 값을 채워 동적으로 프롬프트를 작성 가능함.

In [4]:
template = "안녕하세요는 {country}로 어떻게 말해?"

In [5]:
prompt = PromptTemplate.from_template(template)

In [6]:
english_prompt = prompt.format(country = "영어")
japanes_prompt = prompt.format(country = "일본어")

print(english_prompt)
print(japanes_prompt)

안녕하세요는 영어로 어떻게 말해?
안녕하세요는 일본어로 어떻게 말해?


### 2. 모델 생성 및 호출
1. ChatOpenAI는 openAI로 모델을 선언함. 다양한 모델은 향후 학습
2. .invoke(input) 모델을 호출하는 명령어
2-1. input에 프롬프트를 삽입
2-2. 원하는 모델에 .invoke()사용으로 질문에 대한 답변을 호출

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name = "gpt-4o-mini")


In [8]:
model.invoke(japanes_prompt)

AIMessage(content='안녕하세요는 일본어로 "こんにちは" (곤니치와)라고 말합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 17, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc608545-4c95-41a3-8204-829f83b88201-0', usage_metadata={'input_tokens': 17, 'output_tokens': 19, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 3. Chain



In [9]:
chain = prompt|model

In [ ]:
# 
chain.invoke({"country": "스페인어"})

AIMessage(content='안녕하세요는 스페인어로 "Hola"라고 말합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-8aa30ec8-c63f-493c-aa22-936be39567cf-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain_core.messages import AIMessageChunk
response = chain.stream({"country": "스페인어"})


def stream(response):
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
                answer += token.content
                print(token.content, end="", flush=True)
        elif isinstance(token, str):
                answer += token
                print(token, end="", flush=True)

### 출력 파서 StrOutputParser 결과를 문자열 형식으로 출력

In [11]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


In [12]:
template = """
    너는 question에 상황을 학습용 영어회화를 해주는 어시스턴트야.
    question에 [FORMAT]에 영어회화를 작성해.

    question : {question}

    FORMAT:
    - 영어 회화:
    - 한글 해석:

"""

prompt = PromptTemplate.from_template(template)


In [15]:
chain = prompt|model|output_parser
response = chain.invoke({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."})

stream(response)

- 영어 회화:  
Customer: Hi, I would like to pick up my flight ticket that I reserved.  
Staff: Sure! Can I have your name and reservation number, please?  
Customer: My name is [Your Name], and my reservation number is [Your Reservation Number].  
Staff: Thank you! Here is your ticket. Have a great flight!  

- 한글 해석:  
고객: 안녕하세요, 제가 예약한 비행기표를 받고 싶어요.  
직원: 물론이죠! 이름과 예약 번호를 말씀해 주시겠어요?  
고객: 제 이름은 [당신의 이름]이고, 예약 번호는 [당신의 예약 번호]입니다.  
직원: 감사합니다! 여기 당신의 티켓입니다. 좋은 비행 되세요!  

### 표준 호출
- stream: 응답의 청크를 스트리밍합니다.
- invoke: 입력에 대해 체인을 호출합니다.
- batch: 입력 목록에 대해 체인을 호출합니다.

In [16]:
for token in chain.stream({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."}):
    print(token, end = "",flush=True)

- 영어 회화:  
  A: Hi, I would like to pick up my flight tickets that I reserved.  
  B: Sure! Can I have your name and reservation number, please?  
  A: My name is [Your Name] and the reservation number is [Reservation Number].  
  B: Thank you! Here are your tickets. Have a great flight!

- 한글 해석:  
  A: 안녕하세요, 예약한 비행기표를 받고 싶어요.  
  B: 물론입니다! 이름과 예약 번호를 주시겠어요?  
  A: 제 이름은 [당신의 이름]이고 예약 번호는 [예약 번호]입니다.  
  B: 감사합니다! 여기 당신의 티켓입니다. 좋은 비행 되세요!

In [17]:
chain.invoke({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."})

'- 영어 회화:  \nA: Hi there! How can I help you today?  \nB: Hello! I would like to pick up my flight tickets that I reserved.  \nA: Sure! Can I have your name and reservation number, please?  \nB: Yes, my name is [Your Name] and my reservation number is [Your Reservation Number].  \nA: Thank you! Let me check that for you.  \nB: Great, thank you!\n\n- 한글 해석:  \nA: 안녕하세요! 오늘 무엇을 도와드릴까요?  \nB: 안녕하세요! 예약한 비행기표를 받고 싶습니다.  \nA: 물론입니다! 성함과 예약 번호를 알려주시겠어요?  \nB: 네, 제 이름은 [당신의 이름]이고 예약 번호는 [당신의 예약 번호]입니다.  \nA: 감사합니다! 확인해보겠습니다.  \nB: 좋습니다, 감사합니다!'

In [18]:
chain.batch([{"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."},{"question":"저는 음식점에서 피자를 주문하고 싶어요."}])

['- 영어 회화:  \nA: Hi there! How can I help you today?  \nB: Hello! I would like to pick up my flight ticket that I reserved.  \nA: Sure! Can I have your name and reservation number, please?  \nB: My name is Kim and my reservation number is 123456.  \nA: Thank you, Kim. Here’s your ticket! Have a great flight!  \nB: Thank you very much!\n\n- 한글 해석:  \nA: 안녕하세요! 오늘 무엇을 도와드릴까요?  \nB: 안녕하세요! 예약한 비행기표를 받고 싶어요.  \nA: 물론입니다! 이름과 예약 번호를 알려주시겠어요?  \nB: 제 이름은 김이고 예약 번호는 123456입니다.  \nA: 감사합니다, 김님. 여기 당신의 비행기표입니다! 좋은 비행 되세요!  \nB: 정말 감사합니다!',
 '- 영어 회화:\n  A: Hi there! What would you like to order?\n  B: I would like to order a pizza, please.\n  A: Sure! What toppings would you like on your pizza?\n  B: I’d like pepperoni and mushrooms, please.\n  A: Great choice! Would you like anything to drink with that?\n  B: Yes, I’ll have a soda, please.\n\n- 한글 해석:\n  A: 안녕하세요! 무엇을 주문하시겠어요?\n  B: 피자를 주문하고 싶어요.\n  A: 알겠습니다! 피자에 어떤 토핑을 원하시나요?\n  B: 페퍼로니와 버섯을 원합니다.\n  A: 좋은 선택이에요! 음료수는 무엇을 드릴까요?\n  B: 네, 탄산음료 

### 비동기 호출

In [20]:
async for token in chain.astream({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."}):
    print(token, end = "",flush=True)

- 영어 회화:  
A: Hello! How can I help you today?  
B: Hi, I would like to pick up my flight ticket that I reserved.  
A: Sure! Can I have your name and the reservation number, please?  
B: My name is [Your Name], and the reservation number is [Reservation Number].  
A: Thank you! Let me check that for you. Here’s your ticket. Have a great flight!  

- 한글 해석:  
A: 안녕하세요! 오늘 무엇을 도와드릴까요?  
B: 안녕하세요, 예약한 비행기표를 받고 싶어요.  
A: 물론입니다! 성함과 예약 번호를 말씀해 주시겠어요?  
B: 제 이름은 [당신의 이름]이고, 예약 번호는 [예약 번호]입니다.  
A: 감사합니다! 확인해 볼게요. 여기 당신의 비행기표입니다. 좋은 비행 되세요!

In [22]:
my_process = chain.ainvoke({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."})
await my_process

'- 영어 회화:  \nCustomer: Hi, I would like to pick up my flight tickets that I reserved.  \nStaff: Sure! Can I have your name and reservation number, please?  \nCustomer: My name is [Your Name] and the reservation number is [Your Reservation Number].  \nStaff: Thank you! Let me check that for you. Here are your tickets. Have a great flight!\n\n- 한글 해석:  \n고객: 안녕하세요, 제가 예약한 비행기표를 받고 싶어요.  \n직원: 물론이죠! 성함과 예약 번호를 알려주시겠어요?  \n고객: 제 이름은 [당신의 이름]이고 예약 번호는 [당신의 예약 번호]입니다.  \n직원: 감사합니다! 확인해볼게요. 여기 귀하의 비행기표입니다. 좋은 비행 되세요!'

In [ ]:
my_process = chain.batch([{"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."},{"question":"저는 음식점에서 피자를 주문하고 싶어요."}])
await my_process